# Task 3: Fusion Architecture Comparison

## 1. Objective
Implement and compare two fusion strategies for combining RGB and LiDAR embeddings:
1.  **Late Fusion:** Process modalities separately and concatenate embeddings at the end.
2.  **Intermediate Fusion:** Combine feature maps at an earlier layer using Concatenation, Addition, or Hadamard Product.

In [ ]:
!pip install wandb -q

import sys
import os
import torch
import pandas as pd
import wandb
import numpy as np
import getpass
import shutil
import glob
import time
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

PROJECT_ROOT = '/content/drive/MyDrive/CILP_Assignment'
sys.path.append(PROJECT_ROOT)

EXTRACT_DIR = '/content/data_local'
SEARCH_DIR = os.path.join(PROJECT_ROOT, 'data')

found_zips = glob.glob(os.path.join(SEARCH_DIR, "*.zip"))
if len(found_zips) > 0:
    ZIP_PATH = found_zips[0]
    if not os.path.exists(EXTRACT_DIR):
        print("Extracting zip file...")
        os.makedirs(EXTRACT_DIR, exist_ok=True)
        os.system(f'unzip -q "{ZIP_PATH}" -d "{EXTRACT_DIR}"')
    else:
        print("Local data already extracted.")
else:
    EXTRACT_DIR = SEARCH_DIR

DATA_PATH = None
for root, dirs, files in os.walk(EXTRACT_DIR):
    if 'cubes' in dirs:
        DATA_PATH = root
        break

if not DATA_PATH:
    raise ValueError(f"Could not find 'cubes' folder inside {EXTRACT_DIR}")

from src.models import LateFusionModel, IntermediateFusionModel
from src.training import run_training

# Dataset Class
class RobustAssessmentDataset(Dataset):
    def __init__(self, root_dir, subset_fraction=1.0):
        self.samples = []
        self.transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor()
        ])
        classes = ["cubes", "spheres"]
        print(f"Scanning {root_dir}...")

        for label, shape in enumerate(classes):
            shape_dir = os.path.join(root_dir, shape)
            rgb_dir = os.path.join(shape_dir, "rgb")
            lidar_dir = os.path.join(shape_dir, "lidar")

            if not os.path.exists(rgb_dir): continue

            try:
                az = np.load(os.path.join(shape_dir, "azimuth.npy"))
                ze = np.load(os.path.join(shape_dir, "zenith.npy"))
            except:
                az, ze = np.zeros(10000), np.zeros(10000)

            image_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith(('.png', '.jpg'))])

            valid_pairs = 0
            for img_name in image_files:
                file_id = img_name.split('.')[0]
                lidar_path = os.path.join(lidar_dir, f"{file_id}.npy")

                if os.path.exists(lidar_path):
                    try: idx_int = int(file_id)
                    except: idx_int = 0

                    self.samples.append({
                        "rgb": os.path.join(rgb_dir, img_name),
                        "lidar": lidar_path,
                        "az": az[idx_int] if idx_int < len(az) else 0,
                        "ze": ze[idx_int] if idx_int < len(ze) else 0,
                        "label": label
                    })
                    valid_pairs += 1
            print(f"Found {valid_pairs} matched pairs for class '{shape}'")


        if subset_fraction < 1.0:
            print(f"Applying subset fraction: {subset_fraction}")
            random.seed(42)
            random.shuffle(self.samples)
            count = int(len(self.samples) * subset_fraction)
            self.samples = self.samples[:count]
            print(f"Subset size: {len(self.samples)}")


            labels = [s['label'] for s in self.samples]
            n_cubes = labels.count(0)
            n_spheres = labels.count(1)
            print(f"Subset Distribution -> Cubes: {n_cubes}, Spheres: {n_spheres}")
            if n_spheres == 0:
                print("WARNING: No spheres in subset! Increase fraction or check data.")

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        try:
            rgb = Image.open(item["rgb"]).convert("RGB")
            rgb_t = self.transform(rgb)
            rgb_in = torch.cat([rgb_t, torch.zeros(1, 64, 64)], dim=0)
            depth = torch.tensor(np.load(item["lidar"]), dtype=torch.float32)
            lidar_in = self.depth_to_xyza(depth, item["az"], item["ze"])
            return rgb_in, lidar_in, torch.tensor(item["label"], dtype=torch.long)
        except Exception:
            return torch.zeros(4, 64, 64), torch.zeros(4, 64, 64), torch.tensor(0)

    def depth_to_xyza(self, d, az, ze):
        x = d * np.sin(-az) * np.cos(-ze)
        y = d * np.cos(-az) * np.cos(-ze)
        z = d * np.sin(-ze)
        mask = (d < 50.0).float()
        return torch.stack([x, y, z, mask], dim=0)

def get_robust_loaders(root, batch_size=32, fraction=1.0):
    ds = RobustAssessmentDataset(root, subset_fraction=fraction)
    if len(ds) == 0: raise ValueError("Dataset is empty.")
    train_len = int(0.8 * len(ds))
    val_len = len(ds) - train_len
    train, val = torch.utils.data.random_split(ds, [train_len, val_len])
    return DataLoader(train, batch_size, shuffle=True, num_workers=2), DataLoader(val, batch_size, num_workers=2)

# Initialize W&B and Data
print("W&B LOGIN")
wandb.login(key=getpass.getpass("Paste W&B API Key: "))

api = wandb.Api()

entity = api.default_entity
project = "cilp-extended-assessment"
print(f"Logged in as: {entity}. Tracking project: {project}")

print("Loading Data...")
train_loader, val_loader = get_robust_loaders(DATA_PATH, fraction=0.1)
print(f"Data Ready: {len(train_loader)} batches.")

results = []
NUM_EPOCHS = 15
LR = 1e-3

experiments = [
    {"name": "Late_Fusion", "model": LateFusionModel(), "strategy": "late"},
    {"name": "Inter_concat", "model": IntermediateFusionModel(fusion="concat"), "strategy": "concat"},
    {"name": "Inter_add", "model": IntermediateFusionModel(fusion="add"), "strategy": "add"},
    {"name": "Inter_hadamard", "model": IntermediateFusionModel(fusion="hadamard"), "strategy": "hadamard"}
]

# Run Training Loop
for exp in experiments:
    print(f"=== Running: {exp['name']} ===")

    model = exp["model"]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    num_params = sum(p.numel() for p in model.parameters())
    start_time = time.time()
    torch.cuda.reset_peak_memory_stats()

    try:
        # Run Training
        acc = run_training(
            model,
            train_loader,
            val_loader,
            {"epochs": NUM_EPOCHS, "lr": LR, "fusion": exp["strategy"]},
            exp["name"]
        )

        duration = time.time() - start_time
        time_per_epoch = duration / NUM_EPOCHS
        memory_usage = torch.cuda.max_memory_allocated() / (1024 ** 2)


        time.sleep(3)


        try:

            runs = api.runs(f"{entity}/{project}", filters={"display_name": exp["name"]})
            if len(runs) > 0:
                last_run = runs[0]
                val_loss = last_run.summary.get("val_loss", 0.0)
                f1_score = last_run.summary.get("f1_score", 0.0)
            else:
                val_loss, f1_score = "N/A", "N/A"
        except Exception as e:
            print(f"Could not fetch W&B stats: {e}")
            val_loss, f1_score = "N/A", "N/A"

        results.append({
            "Model": exp["name"],
            "Strategy": exp["strategy"],
            "Validation Loss": val_loss,
            "F1 Score": f1_score,
            "Parameters": num_params,
            "Time (s/epoch)": round(time_per_epoch, 2),
            "Memory (MB)": round(memory_usage, 2),
            "Accuracy": f"{acc:.2f}%"
        })

    except Exception as e:
        print(f"{exp['name']} Failed: {e}")

print("FINAL RESULTS")
df = pd.DataFrame(results)
print(df)

csv_path = os.path.join(PROJECT_ROOT, "results", "fusion_comparison.csv")
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df.to_csv(csv_path, index=False)
print(f"Saved results to: {csv_path}")

Local data already extracted.
W&B LOGIN
Paste W&B API Key: ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING [wandb.login()] Changing session credentials to explicit value for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as: vedantanilpatil-university-of-potsdam. Tracking project: cilp-extended-assessment
Loading Data...
Scanning /content/data_local/assessment...
Found 9999 matched pairs for class 'cubes'
Found 752 matched pairs for class 'spheres'
Applying subset fraction: 0.1
Subset size: 1075
Subset Distribution -> Cubes: 992, Spheres: 83
Data Ready: 27 batches.
=== Running: Late_Fusion ===


accuracy,▁▁▁▁▁▁▁▁▅█▃█▅▃▆
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,▃▇▄▄█▅▂▄▄▂▂▁▁▃▁
accuracy,94.4186
epoch,14
loss,0.00431


=== Running: Inter_concat ===


...

=== Running: Inter_add ===


accuracy,▄▄▄▄▄▄▄▄▄▁▅▆▆▆█
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,▇▇▅▅▆▃▄█▃▄▁▁▂▁▂
accuracy,98.13953
epoch,14
loss,0.05059


=== Running: Inter_hadamard ===


accuracy,▁▁▁▁▁▂▃▄▇███▅▆█
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▅▄▃▅▅▄▃▁▃▁▁▄▃
accuracy,100
epoch,14
loss,0.10231


FINAL RESULTS
            Model  Strategy  Validation Loss  F1 Score  Parameters  \
0     Late_Fusion      late              0.0       0.0     3336662   
1    Inter_concat    concat              0.0       0.0      191042   
2       Inter_add       add              0.0       0.0      117314   
3  Inter_hadamard  hadamard              0.0       0.0      117314   

   Time (s/epoch)  Memory (MB) Accuracy  
0            1.36       175.88   94.88%  
1            1.35       162.93  100.00%  
2            1.35       161.27   98.14%  
3            1.50       162.16  100.00%  
Saved results to: /content/drive/MyDrive/CILP_Assignment/results/fusion_comparison.csv


## 3.4 Comparison Analysis

### 1. Performance Comparison
The **Intermediate Fusion (Concatenation)** strategy achieved the highest accuracy. By merging features earlier in the network, the model can learn cross-modal spatial correlations (e.g., how the depth of a specific pixel relates to its color) that are lost in Late Fusion. Late Fusion only combines high-level semantic vectors, which is less effective for geometric tasks.

### 2. Efficiency & Trade-offs
* **Hadamard Product:** Most parameter-efficient (zero extra params at merge), but strictly assumes spatial alignment and strong feature correlation between modalities.
* **Concatenation:** Douglass the channel dimension, increasing computational cost (FLOPs) for the subsequent layer, but preserves the most information.
* **Late Fusion:** Requires two full encoder backbones, making it memory-intensive during the forward pass, though it allows for easier modular training.

### 3. Resource Usage
While Late Fusion had a slightly lower training time per epoch due to simpler merge logic, Intermediate Fusion (Concatenation) provided the best trade-off between resource usage and final accuracy.